
## Dory Report


In [6]:
import os
import copy
import json
import numpy as np
import pandas as pd
import matplotlib
import ipywidgets as widgets
import matplotlib.pyplot as plt
from datetime import datetime
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Select
from IPython.display import display, display_markdown, HTML

In [7]:
# Define init variable
INPUT_DATA_FN = "output_result.json"


In [8]:
# Result data handling

## load result data file
input_data_fp = os.path.join(os.getcwd(), INPUT_DATA_FN)
with open(input_data_fp) as read_fh:
    full_result_obj = json.load(read_fh)

# get preference name list
preference_name_list = full_result_obj.keys()
    

In [9]:
items_layout = Layout(width='auto')
items_style = {'description_width': 'initial'}
c_slider = Dropdown(layout=items_layout, style=items_style, options=["default"])
d_select = Dropdown(options=preference_name_list, layout=items_layout, style=items_style)


def update_c_range(*args):
    c_slider.options = full_result_obj[d_select.value].keys()

def view3(preference_name='',preference_value=''):
    DATA_LIST = 'time_list'
    DATA_NAME = 'run_time'
    DELETE_KEY = "video-recording-fps"
    HIDE_MISSING_CASE = False
    layout_row = 1
    layout_column = 1
    fig_size_w = 10
    fig_size_h = 5
    matplotlib.rcParams.update({'figure.max_open_warning': 0})

    data_dict = copy.deepcopy(full_result_obj[preference_name])
    for p_value in data_dict:
        if DELETE_KEY in data_dict[p_value]:
            data_dict[p_value].pop(DELETE_KEY)
    #
    # generate merged data
    #
    casename_list = list(set([casename for lable, data in data_dict.items() for casename in data.keys()]))

    case_result = []
    for casename in casename_list:
        result = {
            "casename": casename,
            "result": {}
        }
        for lable, data in data_dict.items():
            result["result"][lable] = {}

        for lable, data in data_dict.items():
            if data.get(casename):
                result["result"][lable][DATA_LIST] = data.get(casename).get(DATA_LIST)
            else:
                result["result"][lable][DATA_LIST] = []
        case_result.append(result)

    #
    # show original data of each cases
    #
    display_markdown('## Merged Result', raw=True)
    display_markdown('Report Generate Time: {}'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')), raw=True)
    display_markdown('{} Cases'.format(len(case_result)), raw=True)
    nightly_better_list = []
    for case in sorted(case_result):
        casename = case.get('casename')
        result = case.get('result')
        d = pd.DataFrame(result)

        # drop empty 'DATA_LIST'
        for c in d:
            if (d[c][DATA_LIST] == []) :
                d.drop(c, axis=1, inplace=True)

        # Retrive the value of 'DATA_NAME' from each run
        value = pd.DataFrame([pd.DataFrame(d[c][DATA_LIST])[DATA_NAME] for c in d]).T
        value.columns = d.columns

        # Plot input latency boxplot
        value_min = min(value.min())
        value_max = max(value.max())
        value_median = pd.DataFrame([pd.DataFrame(d[c][DATA_LIST])[DATA_NAME].median() for c in d]).T
        value_median.columns = d.columns

        # When HIDE_MISSING_CASE is enabled, skip the case if the case data less then input data amount.
        if HIDE_MISSING_CASE and len(INPUT_SOURCE) > len(value.T):
            # display summary
            display_markdown('### {}\nskip draw plot box'.format(casename), raw=True)
            display(HTML(value.describe().to_html()))
        else:

            fig, ax = plt.subplots()
            ax.plot(list(range(1,len(list(value_median.median()))+1)), list(value_median.median()), "r:")

            value.plot.box(layout=(layout_row, layout_column),
                           sharey=True, sharex=True, figsize=(fig_size_w, fig_size_h),
                           ylim=(0, value_max*1.1), ax=ax)
            plt.title(casename)
            if "Nightly" in value_median and "ESR52" in value_median:
                if value_median['Nightly'][0] < value_median['ESR52'][0]:
                    nightly_better_list.append(casename)
            # display summary
            display_markdown('### {}'.format(casename), raw=True)
            display(HTML(value.describe().to_html()))
    # show plot
    plt.show()
    


In [10]:
d_select.observe(update_c_range, 'value')
interactive_obj = interactive(view3, preference_value=c_slider,preference_name=d_select)
interactive_obj

interactive(children=(Dropdown(description=u'preference_name', layout=Layout(width=u'auto'), options=(u'layers.tile-height', u'javascript.options.baselinejit.threshold', u'layers.tile-width', u'javascript.options.ion.threshold'), style=DescriptionStyle(description_width=u'initial'), value=u'layers.tile-height'), Dropdown(description=u'preference_value', layout=Layout(width=u'auto'), options=('default',), style=DescriptionStyle(description_width=u'initial'), value='default'), Output()), _dom_classes=('widget-interact',))